In [ ]:
# !pip install pandas
# !pip install matplotlib
# !pip install seaborn
# !pip install scikit-learn
# !pip install numpy
# !pip install transformers datasets accelerate evaluate sentencepiece protobuf huggingface_hub
# !pip install opendatasets --upgrade


   ----------- ---------------------------- 2/7 [python-slugify]
   ----------------- ---------------------- 3/7 [click]
   ---------------------- ----------------- 4/7 [bleach]
   ---------------------- ----------------- 4/7 [bleach]
   ---------------------- ----------------- 4/7 [bleach]
   ---------------------- ----------------- 4/7 [bleach]
   ---------------------------- ----------- 5/7 [kaggle]
   ---------------------------- ----------- 5/7 [kaggle]
   ---------------------------- ----------- 5/7 [kaggle]
   ---------------------------- ----------- 5/7 [kaggle]
   ---------------------------- ----------- 5/7 [kaggle]
   ---------------------------------- ----- 6/7 [opendatasets]
   ---------------------------------------- 7/7 [opendatasets]



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Core Transformers (models + tokenizers + pipelines)
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, pipeline

# Hugging Face Datasets
from datasets import load_dataset

# Training accelerator (multi-GPU, TPU, CPU support)
from accelerate import Accelerator

# Evaluation metrics
import evaluate

# SentencePiece (used behind the scenes, rarely imported directly)
# import sentencepiece  # usually not needed directly, required internally

# Hugging Face Hub (upload/download models & datasets)
from huggingface_hub import HfApi, notebook_login

import opendatasets as od


c:\Users\thund\Desktop\individual_project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Importing dataset

In [ ]:
# od.download("C:/Users/thund/Downloads/archive/reviews.csv")

ModuleNotFoundError: No module named 'opendatasets.datasets.C:/Users/thund/Downloads/archive/reviews'

In [3]:
reviews_df = pd.read_csv("C:/Users/thund/Desktop/Github/techjam/dataset/reviews.csv")

In [4]:
reviews_df

,business_name,author_name,text,photo,rating,rating_category
0,Haci'nin Yeri - Yigit Lokantasi,Gulsum Akar,We went to Marmaris with my wife for a holiday...,dataset/taste/hacinin_yeri_gulsum_akar.png,5,taste
1,Haci'nin Yeri - Yigit Lokantasi,Oguzhan Cetin,During my holiday in Marmaris we ate here to f...,dataset/menu/hacinin_yeri_oguzhan_cetin.png,4,menu
2,Haci'nin Yeri - Yigit Lokantasi,Yasin Kuyu,Prices are very affordable. The menu in the ph...,dataset/outdoor_atmosphere/hacinin_yeri_yasin_...,3,outdoor_atmosphere
3,Haci'nin Yeri - Yigit Lokantasi,Orhan Kapu,Turkey's cheapest artisan restaurant and its f...,dataset/indoor_atmosphere/hacinin_yeri_orhan_k...,5,indoor_atmosphere
4,Haci'nin Yeri - Yigit Lokantasi,Ozgur Sati,I don't know what you will look for in terms o...,dataset/menu/hacinin_yeri_ozgur_sati.png,3,menu
...,...,...,...,...,...,...
1095,Miss Pizza,Salih Gursoy,There are so many types of pizza; you are surp...,dataset/taste/miss_pizza_salih_gursoy.png,5,taste
1096,Miss Pizza,Kemal Amangeldi,I tried the smoked ribeye pizza; the dough is ...,dataset/indoor_atmosphere/miss_pizza_kemal_ama...,5,indoor_atmosphere
1097,Miss Pizza,Ulkem Esen,Crowded and expensive place.,dataset/menu/miss_pizza_ulkem_esen.png,3,menu
1098,Miss Pizza,Ilkin Saymaz,No bad. It was very crowded; there was no ligh...,dataset/taste/miss_pizza_ilkin_saymaz.png,3,taste


## pseudo labelling 

In [ ]:
# import re

# def rule_based_label(review):
#     if re.search(r"http|www|discount|promo", review.lower()):
#         return "Advertisement"
#     if any(word in review.lower() for word in ["phone", "car", "delivery"]):
#         return "Irrelevant"
#     if "never been here" in review.lower() or "heard it's bad" in review.lower():
#         return "Rant_no_visit"
#     return "Trustworthy"

# df["label"] = df["review"].apply(rule_based_label)


## MODEL

In [6]:
from datasets import Dataset
from transformers import AutoTokenizer

# Example DataFrame
import pandas as pd
df = pd.DataFrame({"review": ["Great food!", "Terrible service."], "label": [1, 0]})
dataset = Dataset.from_pandas(df)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Batched tokenization
def tokenize(batch):
    return tokenizer(batch["review"], padding="max_length", truncation=True)

dataset = dataset.map(tokenize, batched=True)
print(dataset[0])


Map: 100%|██████████| 2/2 [00:00<00:00, 106.34 examples/s]

{'review': 'Great food!', 'label': 1, 'input_ids': [101, 2307, 2833, 999, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [8]:
# ===============================
# 1. Install dependencies (if not already done)
# ===============================
# !pip install transformers datasets accelerate evaluate sentencepiece protobuf huggingface_hub scikit-learn pandas numpy matplotlib seaborn

# ===============================
# 2. Imports
# ===============================
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    pipeline
)
import evaluate
import numpy as np

# ===============================
# 3. Load dataset (example: Google Maps reviews)
# ===============================
# Replace this with your Kaggle/CSV file path
df = pd.read_csv("C:/Users/thund/Desktop/Github/techjam/dataset/reviews.csv")

# For demo, keep only review text
df = df.rename(columns={"text": "review"})  # make sure column is named "review"

# ===============================
# 4. Pseudo-labeling (weak supervision)
# ===============================
import re

def rule_based_label(review):
    text = review.lower()
    if re.search(r"http|www|discount|promo", text):
        return 0  # Advertisement
    if any(word in text for word in ["phone", "car", "delivery"]):
        return 1  # Irrelevant
    if "never been here" in text or "heard it's bad" in text:
        return 2  # Rant_no_visit
    return 3  # Trustworthy

df["label"] = df["review"].astype(str).apply(rule_based_label)

label_names = ["Advertisement", "Irrelevant", "Rant_no_visit", "Trustworthy"]

# ===============================
# 5. Convert to Hugging Face Dataset
# ===============================
dataset = Dataset.from_pandas(df[["review", "label"]])
dataset = dataset.train_test_split(test_size=0.2, seed=42)

# ===============================
# 6. Tokenization
# ===============================
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["review"], padding="max_length", truncation=True)

dataset = dataset.map(tokenize, batched=True)

# ===============================
# 7. Load Model
# ===============================
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=len(label_names)
)

# ===============================
# 8. Training setup
# ===============================
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=preds, references=labels)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# ===============================
# 9. Train
# ===============================
trainer.train()

# ===============================
# 10. Evaluate
# ===============================
results = trainer.evaluate()
print("Evaluation:", results)

# ===============================
# 11. Save Model
# ===============================
trainer.save_model("./trustworthiness_model")
tokenizer.save_pretrained("./trustworthiness_model")

# ===============================
# 12. Run Inference
# ===============================
inference_pipeline = pipeline(
    "text-classification",
    model="./trustworthiness_model",
    tokenizer="./trustworthiness_model",
    return_all_scores=False
)

test_texts = [
    "Never been here but I heard it's bad",
    "50% discount promo!",
    "The service was excellent and food delicious",
]

for text in test_texts:
    pred = inference_pipeline(text)[0]
    print(f"Review: {text}")
    print(f"Prediction: {label_names[int(pred['label'].split('_')[-1])]}")
    print("-" * 30)


Map: 100%|██████████| 220/220 [00:00<00:00, 6412.06 examples/s]
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'